In [15]:
import pandas as pd
import tensorflow as tf

In [16]:
df=pd.read_csv("C:\\Users\\user\\Downloads\\spam.csv",encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [17]:
df2=df[['v1','v2']]

In [18]:
df2.groupby('v1').describe()

v2                                                               
     count unique                                                top freq
v1                                                                       
ham   4825   4516                             Sorry, I'll call later   30
spam   747    653  Please call our customer service representativ...    4

In [19]:
df2['spam']=df2['v1'].apply(lambda x:1 if x=='spam' else 0)

C:\Users\user\anaconda3\envs\pythomopencv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
df2.head()

,v1,v2,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X_train,X_test,y_train,y_test=train_test_split(df2['v2'],df2['spam'],test_size=0.2,stratify=df2['spam']) #stratify used since the data is inbalanced

In [23]:
import tensorflow_hub as hub
import tensorflow_text as text

In [27]:
bert_preprocess=hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder=hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

OSError: SavedModel file does not exist at: C:\Users\user\AppData\Local\Temp\tfhub_modules\602d30248ff7929470db09f7385fc895e9ceb4c0\{saved_model.pbtxt|saved_model.pb}

In [25]:
def get_sentence_embedding(sentences):
    preprocessed_text=bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [26]:
get_sentence_embedding(["500$ discount.hurry up","Bhavin,are you up for a volleyball tomorrow?"])

NameError: name 'bert_preprocess' is not defined

In [ ]:
e=get_sentence_embedding(["banana","grapes","jeff bezos","elon musk"])

In [ ]:
#Use cosine similarity for comparing two vectors

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarity([e[0]],[e[1]]) #0.99 so banana and grape are almost similar

In [ ]:
cosine_similarity([e[0]],[e[2]]) 

In [ ]:
cosine_similarity([e[1]],[e[2]]) 

In [ ]:
cosine_similarity([e[2]],[e[3]]) #0.98 so jeff bezos and elon musk are almost similar

In [ ]:
#Using functional model
text_input=tf.keras.layers.Input(shape=(),dtype=tf.string,name="text")
preprocessed_text=bert_preprocess(text_input)
outputs=bert_encoder(preprocessed_text)
l=tf.keras.layers.Dropout(0.3,name='dropout')(outputs['pooled_output'])
l=tf.keras.layers.Dense(1,activation='sigmoid',name="output")(l)

In [ ]:
model=tf.keras.Model(inputs=[text_input],outputs=[l])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train,epochs=1) #it takes more time,so use gpu in google collab

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
reviews=[
    "Congrats! You won 800 million Dollars",
    "Hey sam,Are you coming for a cricket game tommorow",
    "Why dont you wait till atleast wednesday to see if you get your"
]

In [ ]:
model.predict(reviews) #spam=1 or value>0.5 , not spam <0.5